# Function Calling (Tool Calling): Расширяем возможности LLM

Языковые модели умеют рассуждать, но не умеют **точно считать**, работать с внешними сервисами или получать актуальные данные. Механизм **Function Calling** (или Tool Calling) позволяет модели вызывать внешние функции для выполнения таких задач.

## Как это работает?

```
Пользователь: "Сколько будет 15% от 847?"
       ↓
LLM анализирует запрос
       ↓
LLM решает вызвать функцию: calculate(expression="847 * 0.15")
       ↓
Наш код выполняет вычисление → 127.05
       ↓
Результат отправляется обратно LLM
       ↓
LLM: "15% от 847 равно 127.05"
```

Ключевые моменты:
1. **LLM сама решает**, когда вызвать функцию
2. **LLM формирует аргументы** для вызова на основе запроса пользователя
3. **Наш код** выполняет функцию и возвращает результат
4. **LLM формулирует ответ** на основе результата

Начнём с установки необходимых библиотек:

In [ ]:
%pip install --upgrade openai python-dotenv

**ВНИМАНИЕ**: После установки библиотек рекомендуется перезапустить Kernel ноутбука.

In [ ]:
!curl -o .env {{url_of_dotenv_file}}


In [1]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

folder_id = os.environ["folder_id"]
api_key = os.environ["api_key"]

def printx(string):
    display(Markdown(string))

print(f"✅ Авторизация настроена (folder_id: {folder_id[:8]}...)")

✅ Авторизация настроена (folder_id: b1gbicod...)


---

## Часть 1: Настройка OpenAI клиента

Создадим клиент для работы с Yandex Cloud через OpenAI-совместимый API:

In [2]:
from openai import OpenAI

model = f"gpt://{folder_id}/qwen3-235b-a22b-fp8/latest"

client = OpenAI(
    base_url="https://ai.api.cloud.yandex.net/v1",
    api_key=api_key,
    project=folder_id
)

print("✅ Клиент OpenAI настроен")

✅ Клиент OpenAI настроен


---

## Часть 2: Описание инструмента — Калькулятор

Чтобы LLM могла вызвать функцию, нам нужно описать её в специальном формате — **JSON Schema**. Модель использует это описание, чтобы понять:
- Что делает функция
- Какие параметры принимает
- Какого типа должны быть параметры

Создадим простой калькулятор, который умеет вычислять математические выражения:

In [3]:
# Описание инструмента для Function Calling
calculator_tools = [
    {
        "type": "function",
        "name": "calculate",
        "description": "Вычислить математическое выражение. Поддерживает арифметические операции (+, -, *, /), возведение в степень (**), корень (sqrt), тригонометрию (sin, cos, tan), логарифмы (log, log10) и константы (pi, e).",
        "parameters": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "Математическое выражение на Python, например: '2**10', 'sqrt(144)', 'sin(pi/4)'"
                }
            },
            "required": ["expression"]
        }
    }
]

print("✅ Инструмент 'calculate' определён")
print("\nJSON-схема:")
print(json.dumps(calculator_tools[0], indent=2, ensure_ascii=False))

✅ Инструмент 'calculate' определён

JSON-схема:
{
  "type": "function",
  "name": "calculate",
  "description": "Вычислить математическое выражение. Поддерживает арифметические операции (+, -, *, /), возведение в степень (**), корень (sqrt), тригонометрию (sin, cos, tan), логарифмы (log, log10) и константы (pi, e).",
  "parameters": {
    "type": "object",
    "properties": {
      "expression": {
        "type": "string",
        "description": "Математическое выражение на Python, например: '2**10', 'sqrt(144)', 'sin(pi/4)'"
      }
    },
    "required": [
      "expression"
    ]
  }
}


---

## Часть 3: Реализация функции калькулятора

Теперь реализуем саму функцию. Мы используем `math` для безопасных вычислений:

In [4]:
import math

def calculate(expression: str) -> str:
    """
    Безопасно вычисляет математическое выражение.
    
    Использует ограниченный набор функций из модуля math,
    чтобы избежать выполнения произвольного кода.
    """
    # Словарь разрешённых функций и констант
    allowed = {
        "sqrt": math.sqrt,
        "sin": math.sin,
        "cos": math.cos,
        "tan": math.tan,
        "log": math.log,
        "log10": math.log10,
        "log2": math.log2,
        "exp": math.exp,
        "abs": abs,
        "round": round,
        "pi": math.pi,
        "e": math.e,
    }
    
    try:
        # Вычисляем выражение в ограниченном окружении
        result = eval(expression, {"__builtins__": {}}, allowed)
        return str(result)
    except Exception as e:
        return f"Ошибка вычисления: {e}"


# Тестируем калькулятор
print("Тесты калькулятора:")
print(f"  2 + 2 = {calculate('2 + 2')}")
print(f"  sqrt(144) = {calculate('sqrt(144)')}")
print(f"  sin(pi/2) = {calculate('sin(pi/2)')}")
print(f"  2**10 = {calculate('2**10')}")
print(f"  log(e) = {calculate('log(e)')}")

Тесты калькулятора:
  2 + 2 = 4
  sqrt(144) = 12.0
  sin(pi/2) = 1.0
  2**10 = 1024
  log(e) = 1.0


---

## Часть 4: Первый вызов с Function Calling

Отправим запрос модели с описанием инструмента и посмотрим, что она вернёт. Модель **не выполняет** функцию сама — она лишь сообщает, какую функцию хочет вызвать и с какими аргументами.

In [6]:
instruction = """
Ты — математический ассистент. У тебя есть калькулятор для точных вычислений.
Когда пользователь задаёт задачу, требующую вычислений:
1. Сформулируй математическое выражение
2. Вызови функцию calculate для получения точного результата
3. Объясни ответ пользователю

Всегда используй калькулятор для вычислений — не считай в уме!
"""

# Отправляем запрос
res = client.responses.create(
    model=model,
    tools=calculator_tools,
    instructions=instruction,
    input="Чему равен квадратный корень из 2026?"
)

# Смотрим, что вернула модель
print("Ответ модели содержит:")
for item in res.output:
    print(f"  - тип: {item.type}")
    if item.type == "function_call":
        print(f"    функция: {item.name}")
        print(f"    аргументы: {item.arguments}")
        print(f"    call_id: {item.call_id}")

Ответ модели содержит:
  - тип: function_call
    функция: calculate
    аргументы: {"expression": "sqrt(2026)"}
    call_id: chatcmpl-tool-913a4f9300704d3f8405c1b3e6937d58


Модель вернула `function_call` — это значит, она хочет вызвать нашу функцию. Теперь нам нужно:
1. Выполнить функцию с указанными аргументами
2. Вернуть результат модели
3. Получить финальный ответ

---

## Часть 5: Полный цикл — от запроса до ответа

Соберём весь процесс в одну функцию. Цикл работает так:

```
while есть function_call в ответе:
    1. Выполнить каждую запрошенную функцию
    2. Отправить результаты модели
    3. Получить новый ответ
```

In [7]:
def ask_with_calculator(prompt: str, max_iterations: int = 10):
    """
    Отправляет вопрос модели с доступом к калькулятору.
    Автоматически обрабатывает вызовы функций в цикле.
    """
    print(f"❓ Вопрос: {prompt}")
    print("=" * 50)
    
    # Первый запрос к модели
    res = client.responses.create(
        model=model,
        tools=calculator_tools,
        instructions=instruction,
        store=True,
        input=prompt
    )
    
    iteration = 0
    
    while iteration < max_iterations:
        # Ищем вызовы функций в ответе
        tool_calls = [item for item in res.output if item.type == "function_call"]
        
        if not tool_calls:
            # Нет вызовов функций — модель готова дать ответ
            break
        
        # Обрабатываем каждый вызов функции
        outputs = []
        for call in tool_calls:
            args = json.loads(call.arguments) if call.arguments else {}
            expression = args.get("expression", "")
            
            # Выполняем вычисление
            result = calculate(expression)
            print(f"  🔢 calculate({expression}) = {result}")
            
            # Формируем ответ для модели
            outputs.append({
                "type": "function_call_output",
                "call_id": call.call_id,
                "output": result
            })
        
        # Отправляем результаты обратно модели
        res = client.responses.create(
            model=model,
            tools=calculator_tools,
            previous_response_id=res.id,
            store=True,
            input=outputs
        )
        
        iteration += 1
    
    print("=" * 50)
    printx(res.output_text)

### Тест: простое вычисление

In [8]:
ask_with_calculator("Чему равен квадратный корень из 2026?")

❓ Вопрос: Чему равен квадратный корень из 2026?
  🔢 calculate(sqrt(2026)) = 45.0111097397076


Квадратный корень из 2026 приблизительно равен 45.011.

### Тест: текстовая задача

LLM должна сама понять, какое выражение нужно составить:

In [9]:
ask_with_calculator(
    "В лаборатории 12 исследователей. Каждый проводит по 3 эксперимента в день. "
    "Эксперимент длится 45 минут. Сколько часов в сумме занимают все эксперименты за 5 рабочих дней?"
)

❓ Вопрос: В лаборатории 12 исследователей. Каждый проводит по 3 эксперимента в день. Эксперимент длится 45 минут. Сколько часов в сумме занимают все эксперименты за 5 рабочих дней?
  🔢 calculate((12 * 3 * 5 * 45) / 60) = 135.0


Общее время, которое занимают все эксперименты за 5 рабочих дней, составляет **135 часов**.

### Тест: несколько шагов вычислений

In [11]:
ask_with_calculator(
    "Я написал статью на 24 страницы. Рецензент попросил сократить её на 30%. "
    "Сколько страниц останется? А если потом редактор попросит добавить ещё 15% к сокращённой версии?"
)

❓ Вопрос: Я написал статью на 24 страницы. Рецензент попросил сократить её на 30%. Сколько страниц останется? А если потом редактор попросит добавить ещё 15% к сокращённой версии?
  🔢 calculate(24 * (1 - 0.3)) = 16.799999999999997
  🔢 calculate(24 * (1 - 0.3) * (1 + 0.15)) = 19.319999999999997


После сокращения статьи на 30% останется примерно **16.8 страниц**.

Если затем добавить 15% к сокращённой версии, получится около **19.32 страниц**.

Таким образом:
- После сокращения: **16.8 страниц**
- После добавления 15%: **19.32 страниц**

---

## Часть 6: Использование Pydantic для описания функций

В предыдущем примере мы описывали функцию вручную в виде JSON-схемы. Для больших проектов удобнее использовать **Pydantic** — библиотеку для валидации данных, которая автоматически генерирует схему из типов Python.

Преимущества:
- Меньше ручного кода
- Автоматическая валидация входных данных
- Код и схема всегда синхронизированы

In [12]:
from pydantic import BaseModel, Field


class Calculate(BaseModel):
    """Вычислить математическое выражение. Поддерживает арифметику, корни, тригонометрию, логарифмы."""
    expression: str = Field(
        description="Математическое выражение на Python, например: '2**10', 'sqrt(144)', 'sin(pi/4)'"
    )
    
    def execute(self) -> str:
        return calculate(self.expression)


TOOLS = [Calculate]

# Посмотрим на автоматически сгенерированную JSON-схему
print("JSON-схема из Pydantic:")
print(json.dumps(Calculate.model_json_schema(), indent=2, ensure_ascii=False))

JSON-схема из Pydantic:
{
  "description": "Вычислить математическое выражение. Поддерживает арифметику, корни, тригонометрию, логарифмы.",
  "properties": {
    "expression": {
      "description": "Математическое выражение на Python, например: '2**10', 'sqrt(144)', 'sin(pi/4)'",
      "title": "Expression",
      "type": "string"
    }
  },
  "required": [
    "expression"
  ],
  "title": "Calculate",
  "type": "object"
}


---

## Часть 7: Универсальный класс Agent

Создадим универсальный класс `Agent`, который:
- Автоматически генерирует описания инструментов из Pydantic-моделей
- Обрабатывает вызовы функций в цикле
- Может использоваться с любыми наборами инструментов

In [13]:
class Agent:
    """
    Универсальный агент с поддержкой Function Calling.
    Принимает список Pydantic-моделей как инструменты.
    """
    
    def __init__(self, instruction: str, tools: list, model: str = model, max_iterations: int = 20):
        self.instruction = instruction
        self.model = model
        self.max_iterations = max_iterations
        
        # Создаём словарь имя -> класс для поиска
        self.tool_map = {cls.__name__: cls for cls in tools}
        
        # Генерируем описания инструментов для API
        self.tools_schema = [
            {
                "type": "function",
                "name": cls.__name__,
                "description": cls.__doc__ or "",
                "parameters": cls.model_json_schema()
            }
            for cls in tools
        ]
    
    def __call__(self, message: str, verbose: bool = True) -> str:
        """
        Обработка сообщения с автоматическим выполнением функций.
        """
        # Первый запрос
        res = client.responses.create(
            model=self.model,
            tools=self.tools_schema,
            instructions=self.instruction,
            store=True,
            input=message
        )
        
        # Цикл обработки функций
        for _ in range(self.max_iterations):
            tool_calls = [item for item in res.output if item.type == "function_call"]
            
            if not tool_calls:
                break
            
            outputs = []
            for call in tool_calls:
                try:
                    cls = self.tool_map[call.name]
                    args = json.loads(call.arguments) if call.arguments else {}
                    obj = cls.model_validate(args)
                    result = obj.execute()
                    if verbose:
                        print(f"  🔧 {call.name}({call.arguments}) → {result}")
                except Exception as e:
                    result = f"Ошибка: {e}"
                
                outputs.append({
                    "type": "function_call_output",
                    "call_id": call.call_id,
                    "output": result
                })
            
            res = client.responses.create(
                model=self.model,
                tools=self.tools_schema,
                previous_response_id=res.id,
                store=True,
                input=outputs
            )
        
        return res.output_text or ""

print("✅ Класс Agent определён")

✅ Класс Agent определён


### Тест: Agent с калькулятором

In [14]:
math_agent = Agent(
    instruction=instruction,
    tools=TOOLS
)

result = math_agent("Если каждый из 8 GPU обрабатывает 25000 токенов в секунду, сколько токенов обработают все GPU за 12 часов?")
printx(result)

  🔧 Calculate({"expression": "200000 * 43200"}) → 8640000000


Все 8 GPU обработают **8 640 000 000 токенов** за 12 часов.

In [15]:
result = math_agent("Чему равно e в степени π минус π в степени e?")
printx(result)

  🔧 Calculate({"expression": "exp(pi) - pi**exp(1)"}) → 0.6815349144182257


Значение выражения $ e^{\pi} - \pi^{e} $ приблизительно равно **0.6815**.

---

## Выводы

В этом ноутбуке мы разобрали механику **Function Calling**:

1. **Описание функции** — через JSON-схему или Pydantic-модель
2. **Цикл вызовов** — LLM запрашивает вызов → мы выполняем → возвращаем результат
3. **Универсальный Agent** — класс, который автоматизирует весь процесс

Этот же паттерн применим к любым инструментам: поиск, базы данных, API сервисов.

---

## Упражнение для самостоятельной работы

Попробуйте добавить ещё один инструмент — **конвертер единиц измерения** для научных вычислений. Например:

```python
class ConvertUnits(BaseModel):
    """Конвертировать значение из одной единицы измерения в другую."""
    value: float = Field(description="Значение для конвертации")
    from_unit: str = Field(description="Исходная единица (например: km, m, miles, celsius, kelvin, eV, joules)")
    to_unit: str = Field(description="Целевая единица")
    
    def execute(self) -> str:
        # Реализуйте конвертацию здесь
        ...
```

Подсказки:
- Создайте словарь коэффициентов пересчёта
- Добавьте `ConvertUnits` в список `TOOLS`
- Протестируйте: `"Какова энергия фотона с длиной волны 500 нм в электронвольтах?"`